<a href="https://colab.research.google.com/github/ColdTbrew/pixel_Satellite-Image-Segmentation/blob/main/infer_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir test
%cd test

/content/test


In [4]:
cp -rvf /content/drive/MyDrive/copy .

'/content/drive/MyDrive/copy' -> './copy'
'/content/drive/MyDrive/copy/test_image.tar' -> './copy/test_image.tar'
'/content/drive/MyDrive/copy/best_mIoU_iter_27200.pth' -> './copy/best_mIoU_iter_27200.pth'
'/content/drive/MyDrive/copy/sample_submission.csv' -> './copy/sample_submission.csv'
'/content/drive/MyDrive/copy/swin_v2.py' -> './copy/swin_v2.py'
'/content/drive/MyDrive/copy/infer.py' -> './copy/infer.py'
'/content/drive/MyDrive/copy/test.csv' -> './copy/test.csv'


In [5]:
!tar xvf "./copy/test_image.tar"

In [6]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 9.6 MB/s eta 0:00:00


In [7]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!git checkout main
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 15908, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 15908 (delta 94), reused 166 (delta 56), pack-reused 15633
Receiving objects: 100% (15908/15908), 23.01 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (11112/11112), done.
/content/test/mmsegmentation
Already on 'main'
Your branch is up to date with 'origin/main'.
Obtaining file:///content/test/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation
/content/test


In [12]:
%cd /content/test/mmsegmentation
!pwd

/content/test/mmsegmentation
/content/test/mmsegmentation


In [ ]:
import os
import mmcv
import torch
from tqdm.notebook import tqdm
from PIL import Image
import pandas as pd
import numpy as np

from mmseg.apis import init_model, inference_model

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def main():
    save_dir = '/content/drive/MyDrive/csvs'  # save dir
    file_name = 'submit_swin'  # 저장할 파일 이름
    config_path = '/content/test/copy/swin_v2.py'  # config 파일 위치
    ckpt_path = '/content/test/copy/best_mIoU_iter_27200.pth' # 체크포인트 파일 위치
    sample_path = "/content/test/copy/test.csv"
    submit_path = "/content/test/copy/sample_submission.csv"
    test_image_path = "/content/test/test_image"
    device = "cuda"

    model = init_model(config_path, ckpt_path, device)

    data = pd.read_csv(sample_path)['id'].values.tolist()

    with torch.no_grad():
        model.eval()
        result = []
        for img_id in tqdm(data):
            img_path = os.path.join(test_image_path, img_id + ".png")
            mask = inference_model(model, img_path)
            mask = mask.pred_sem_seg.data
            mask = torch.squeeze(mask).cpu().numpy()
            mask = mask.astype(np.uint8)
            mask = Image.fromarray(mask)
            mask = mask.resize((960, 540), Image.NEAREST)
            mask = np.array(mask)
            for class_id in range(12):
                class_mask = (mask == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0:  # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else:  # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

    submit = pd.read_csv(submit_path)
    submit['mask_rle'] = result
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    submit.to_csv(os.path.join(save_dir, file_name + '.csv'), index=False)
main()


/content/test/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/test/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /content/test/copy/best_mIoU_iter_27200.pth


  0%|          | 0/1898 [00:00<?, ?it/s]